In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
#import libraries and some constants

import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LogNorm
import pandas as pd
import uproot as ur
import atlas_mpl_style as ampl
ampl.use_atlas_style()

path_prefix = '/g/g16/heinz6/workspace/projects/caloml-atlas/code/caloml-atlas-py/'
plotpath = path_prefix+'regression/Plots/'
modelpath = path_prefix+'regression/Models/'

In [7]:
# import our resolution utilities

import sys
sys.path.append(path_prefix)
from  util import resolution_util as ru
from  util import plot_util as pu
from  util import ml_util as mu

Using TensorFlow backend.


In [8]:
inputpath = '/usr/WS1/hip/ML4Jets/regression_images/sample_jet_events/'
rootfiles = ["mltree.pool"]
branches = ['TileBar2', 'EMB3', 'cluster_ENG_CALIB_TOT', 'cluster_sumCellE',\
            'TileBar1', 'TileBar0', 'cluster_nCells', 'clusterPt', 'clusterIndex',\
            'clusterEta', 'EMB1', 'clusterPhi', 'EMB2', 'clusterE', 'truthE']

trees, pdata = mu.setupPionData(inputpath, rootfiles, branches)

In [9]:
pdata['mltree.pool']

,TileBar2[0][0],TileBar2[0][1],TileBar2[0][2],TileBar2[0][3],TileBar2[1][0],TileBar2[1][1],TileBar2[1][2],TileBar2[1][3],EMB3[0][0],EMB3[0][1],...,EMB2[15][8],EMB2[15][9],EMB2[15][10],EMB2[15][11],EMB2[15][12],EMB2[15][13],EMB2[15][14],EMB2[15][15],clusterE,truthE
entry,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.001884,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.007009,6500.0
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.006814,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.435404,6500.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.576868,6500.0
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.415128,6500.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.225583,6500.0
5,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.201715,6500.0
6,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.740174,6500.0
7,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.671736,6500.0
8,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.624033,6500.0


In [10]:
pdata['mltree.pool'].keys()

Index(['TileBar2[0][0]', 'TileBar2[0][1]', 'TileBar2[0][2]', 'TileBar2[0][3]',
       'TileBar2[1][0]', 'TileBar2[1][1]', 'TileBar2[1][2]', 'TileBar2[1][3]',
       'EMB3[0][0]', 'EMB3[0][1]',
       ...
       'EMB2[15][8]', 'EMB2[15][9]', 'EMB2[15][10]', 'EMB2[15][11]',
       'EMB2[15][12]', 'EMB2[15][13]', 'EMB2[15][14]', 'EMB2[15][15]',
       'clusterE', 'truthE'],
      dtype='object', length=945)

In [11]:
pcells = {
    ifile : {
        layer : mu.setupCells(itree, layer, flatten = False)
        for layer in mu.cell_meta
    }
    for ifile, itree in trees.items()
}

In [12]:
keys = trees['mltree.pool'].keys()
keys

[b'runNumber',
 b'eventNumber',
 b'truthE',
 b'truthPt',
 b'truthEta',
 b'truthPhi',
 b'clusterIndex',
 b'nCluster',
 b'clusterE',
 b'clusterECalib',
 b'clusterPt',
 b'clusterEta',
 b'clusterPhi',
 b'cluster_nCells',
 b'cluster_sumCellE',
 b'cluster_ENG_CALIB_TOT',
 b'cluster_ENG_CALIB_OUT_T',
 b'cluster_ENG_CALIB_DEAD_TOT',
 b'cluster_EM_PROBABILITY',
 b'cluster_HAD_WEIGHT',
 b'cluster_OOC_WEIGHT',
 b'cluster_DM_WEIGHT',
 b'cluster_CENTER_MAG',
 b'cluster_FIRST_ENG_DENS',
 b'cluster_ENERGY_DigiHSTruth',
 b'cluster_cell_dR_min',
 b'cluster_cell_dR_max',
 b'cluster_cell_dEta_min',
 b'cluster_cell_dEta_max',
 b'cluster_cell_dPhi_min',
 b'cluster_cell_dPhi_max',
 b'cluster_cell_centerCellEta',
 b'cluster_cell_centerCellPhi',
 b'cluster_cell_centerCellLayer',
 b'cluster_cellE_norm',
 b'EMB1',
 b'EMB2',
 b'EMB3',
 b'TileBar0',
 b'TileBar1',
 b'TileBar2',
 b'duplicate_EMB1',
 b'duplicate_EMB2',
 b'duplicate_EMB3',
 b'duplicate_TileBar0',
 b'duplicate_TileBar1',
 b'duplicate_TileBar2']

In [13]:
event_trees = {
        rfile: ur.open(inputpath+rfile+".root")['EventTree']
        for rfile in rootfiles
}

In [14]:
event_keys = event_trees['mltree.pool'].keys()
event_keys

[b'runNumber',
 b'eventNumber',
 b'lumiBlock',
 b'coreFlags',
 b'mcEventNumber',
 b'mcChannelNumber',
 b'mcEventWeight',
 b'nTruthPart',
 b'pdgId',
 b'status',
 b'barcode',
 b'truthPartPt',
 b'truthPartE',
 b'truthPartMass',
 b'truthPartEta',
 b'truthPartPhi',
 b'clusterCount',
 b'nCluster',
 b'clusterE',
 b'clusterPt',
 b'clusterEta',
 b'clusterPhi',
 b'cluster_nCells',
 b'cluster_sumCellE',
 b'cluster_cell_dEta',
 b'cluster_cell_dPhi',
 b'cluster_cell_dR_min',
 b'cluster_cell_dR_max',
 b'cluster_cell_dEta_min',
 b'cluster_cell_dEta_max',
 b'cluster_cell_dPhi_min',
 b'cluster_cell_dPhi_max',
 b'm_cluster_cell_centerCellEta',
 b'm_cluster_cell_centerCellPhi',
 b'm_cluster_cell_centerCellLayer',
 b'AntiKt4EMTopoJetsPt',
 b'AntiKt4EMTopoJetsEta',
 b'AntiKt4EMTopoJetsPhi',
 b'AntiKt4EMTopoJetsE',
 b'AntiKt4LCTopoJetsPt',
 b'AntiKt4LCTopoJetsEta',
 b'AntiKt4LCTopoJetsPhi',
 b'AntiKt4LCTopoJetsE',
 b'AntiKt4TruthJetsPt',
 b'AntiKt4TruthJetsEta',
 b'AntiKt4TruthJetsPhi',
 b'AntiKt4TruthJetsE

In [15]:
pcells['mltree.pool']['TileBar2'].shape

(159888, 2, 4)

In [47]:
inputpath2 = '/g/g16/heinz6/workspace/projects/caloml-atlas/code/atlasml-karande-heinz/store/inference_jet/'

prediction = np.load(inputpath2+'inference_data_log_mae_with_fold_0_best.npz')

calib_pred = prediction['calib_pred']
calib = prediction['calib']
label_pred = prediction['label_pred']

In [48]:
print('Number of clusters: {}'.format(len(label_pred)))
print('Number of pi0: {}'.format(sum(label_pred==0)))
print('Number of pi+/-: {}'.format(sum(label_pred==1)))
print('Percentage of pi+/-: {}'.format(sum(label_pred==1)/len(label_pred)))

Number of clusters: 98581
Number of pi0: 880
Number of pi+/-: 97701
Percentage of pi+/-: 0.9910733305606557


In [49]:
calib_ratio = calib_pred/calib

print(np.median(calib_ratio))

300.97305655416415


In [50]:
prediction2 = np.load(inputpath2+'inference_data_v6_log_mae_with_fold_0_best.npz')

calib_pred2 = prediction2['calib_pred']
calib2 = prediction2['calib']
label_pred2 = prediction2['label_pred']

print('Number of clusters: {}'.format(len(label_pred2)))
print('Number of pi0: {}'.format(sum(label_pred2==0)))
print('Number of pi+/-: {}'.format(sum(label_pred2==1)))
print('Percentage of pi+/-: {}'.format(sum(label_pred2==1)/len(label_pred2)))

calib_ratio2 = calib_pred2/calib2

print(np.median(calib_ratio2))

Number of clusters: 347376
Number of pi0: 10805
Number of pi+/-: 336571
Percentage of pi+/-: 0.9688953756160472
1.0027961403066337


In [ ]:
import numpy as np
import tensorflow as tf
import sys

sys.path.append('/g/g16/heinz6/workspace/projects/caloml-atlas/code/atlasml-karande-heinz/')
from util.ml_util import rebinUp

class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, tree, batch_size=32, nonempty = True, noncalib_zero = True, shuffle=False):
        'Initialization'
        self.tree = tree
        self.branches = tree.keys()
        if len(self.branches) == 0:
            print('Error: tree is empty.')
            sys.exit()
        self.img_branches = ['EMB1', 'EMB2', 'EMB3', 'TileBar0', 'TileBar1', 'TileBar2',\
                            'cluster_ENG_CALIB_TOT','clusterE']
        self.num_clusters = len(tree.lazyarray(branches[0]))
        
        self.inds = np.arange(self.num_clusters)
        self.inds_nonempty = self.inds
        self.inds_calib_nonzero = self.inds
        
        self.batch_size = batch_size
        if self.batch_size <= 0:
            print('Error: invalid batch_size.')
            sys.exit()
            
        self.nonempty = nonempty
        self.noncalib_zero = noncalib_zero
        self.shuffle = shuffle
        self.on_epoch_end()
        
        # if self.nonempty = False:
        #     self.__filter_empty_images()
        # if self.noncalib_zero = False:
        #     self.__filter_calib_zero_images()
        # 
        # self.__intersect_indices()
        print('\nImages are clean.')

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.num_clusters / self.batch_size))

    def __getitem__(self, idx):
        'Generate one batch of data'
        # Generate indexes of the batch
        temp_inds = self.inds[idx*self.batch_size:(idx+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(idx)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, idx):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        data = self.tree.iterate(branches = self.img_branches, entrystart = idx*self.batch_size,
                                 entrystop = (idx+1)*self.batch_size)
        
        calib = data['cluster_ENG_CALIB_TOT']
        clusterE = data['clusterE']
        # emb1, emb2, emb3, tile0, tile1, tile2 = self.unscale_images(data, clusterE)
        emb1 = data['EMB1']
        emb2 = data['EMB2']
        emb3 = data['EMB3']
        tile0 = data['TileBar0']
        tile1 = data['TileBar1']
        tile2 = data['TileBar2']

        emb3 = self.rebinUpCluster(emb3, 16, 16)
        tile2 = self.rebinUpCluster(tile2, 4, 4)

        # Stacking and adding axis as required
        emb1 = np.expand_dims(emb1, axis=3)
        emb23 = np.stack([emb2, emb3], axis=3)
        tiles = np.stack([tile0, tile1, tile2], axis=3)
        
        X = [emb1,emb23,tiles]
        y = calib

        return X, y
    
    def __filter_empty_images(self):
    
    def __filter_calib_zero_images(self):
    
    def __intersect_indices(self):
        inds_set = set(self.inds)
        inds_nonempty_set = set(self.inds_nonempty)
        inds_calib_nonzero_set = set(self.inds_calib_nonzero)
        inds_set = set.intersection(inds_set, inds_nonempty_set, inds_calib_nonzero_set)
        self.inds = np.array(list(inds_set))
        
    def unscale_images(self, data, clusterE):
        if (len(data['EMB1']) != len(clusterE)):
            print('Input lengths do not match... exiting')
            sys.exit()

        emb1 = np.einsum('ijk,i->ijk', data['EMB1'], clusterE)
        emb2 = np.einsum('ijk,i->ijk', data['EMB2'], clusterE)
        emb3 = np.einsum('ijk,i->ijk', cell_data['EMB3'], clusterE)

        tile0 = np.einsum('ijk,i->ijk', data['TileBar0'], clusterE)
        tile1 = np.einsum('ijk,i->ijk', data['TileBar1'], clusterE)
        tile2 = np.einsum('ijk,i->ijk', data['TileBar2'], clusterE)

        return emb1, emb2, emb3, tile0, tile1, tile2
    
    def rebinUpCluster(self, clust, targetEta, targetPhi):
        clust = [rebinUp(c, targetEta, targetPhi) for c in clust]
        clust = np.array(clust)
        return clust

In [10]:
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow import keras as keras
from keras.wrappers.scikit_learn import KerasRegressor

In [11]:
import tensorflow as tf

gpu_list = ["/gpu:0", "/gpu:1"]
# strategy = tf.distribute.MirroredStrategy(devices=gpu_list)
# strategy = tf.distribute.MirroredStrategy()
# ngpu = strategy.num_replicas_in_sync
ngpu = 1
print ('Number of devices: {}'.format(ngpu))

Number of devices: 1


In [13]:
training_classes = ['mltree.pool']
# create train/validation/test subsets containing 70%/10%/20%
# of events from each type of pion event
pp, pp_cells, pp_label = mu.createTrainingDatasets(training_classes, pdata, pcells)

In [15]:
pp_cells['EMB1']

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       ...,

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0.